In [1]:
import pandas as pd
import numpy as np

# Single Dataset Creation

In [2]:
excel_file = pd.ExcelFile("DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx")
excel_file.sheet_names

['SURESTE',
 'NORESTE',
 'CENTRO',
 'NOROESTE',
 'SUROESTE',
 'NOROESTE2',
 'NORTE',
 'SUROESTE2',
 'SURESTE2',
 'SURESTE3',
 'SUR',
 'NORTE2',
 'NORESTE2',
 'NORESTE3',
 'NOROESTE3',
 'CATÁLOGO']

In [3]:
dataset = pd.concat(
    (excel_file.parse(sheet).assign(Estacion=i) for i, sheet in enumerate(excel_file.sheet_names[:-1])),
    ignore_index=True
)

In [4]:
dataset.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,Estacion
0,2022-01-01 00:00:00,2.36,19.8,48.4,68.0,5.0,142.0,111.00,715.7,0.0,59.0,4.0,0.0,21.65,6.5,285.0,0
1,2022-01-01 01:00:00,2.29,10.8,44.5,55.1,6.0,168.0,112.16,715.5,0.0,59.0,4.6,0.0,21.21,7.1,284.0,0
2,2022-01-01 02:00:00,2.71,28.3,47.1,75.2,4.0,203.0,139.31,715.4,0.0,62.0,5.9,0.0,21.21,7.7,290.0,0
3,2022-01-01 03:00:00,2.31,19.6,42.1,61.5,5.0,262.0,177.68,715.1,0.0,59.0,5.5,0.0,20.44,8.2,294.0,0
4,2022-01-01 04:00:00,1.85,13.0,39.5,52.5,4.0,185.0,134.21,715.1,0.0,59.0,4.9,0.0,19.80,8.1,287.0,0


In [5]:
rubric = excel_file.parse(excel_file.sheet_names[-1])
rubric['Nombre_Estacion'] = rubric['Nombre_Estacion'].str.upper()
rubric = rubric.reset_index()
rubric.head()

,index,Nombre_Estacion,Clave_Estacion,location
0,0,NORESTE3 PESQUERIA,Pesqueria,"25.791343,-100.078176"
1,1,NORTE2 UNIVERSIDAD,Universidad,"25.729787,-100.310028"
2,2,NOROESTE SAN BERNABÉ,San Bernabé,"25.75712,-100.365974"
3,3,NOROESTE 2 GARCIA,Garcia,"25.783331,-100.585833"
4,4,CENTRO OBISPADO,Obispado,"25.67602,-100.335847"


In [6]:
data_joined = dataset.merge(rubric, left_on='Estacion', right_on='index', how='left')
data_joined.drop(columns=['index', 'Estacion'], inplace=True)
data_joined[['x_coord', 'y_coord']] = data_joined['location'].str.split(',', expand=True)
data_joined['x_coord'] = data_joined['x_coord'].astype(float)
data_joined['y_coord'] = data_joined['y_coord'].astype(float)
data_joined.drop(columns=['location'], inplace=True)
data_joined.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,Nombre_Estacion,Clave_Estacion,x_coord,y_coord
0,2022-01-01 00:00:00,2.36,19.8,48.4,68.0,5.0,142.0,111.00,715.7,0.0,59.0,4.0,0.0,21.65,6.5,285.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
1,2022-01-01 01:00:00,2.29,10.8,44.5,55.1,6.0,168.0,112.16,715.5,0.0,59.0,4.6,0.0,21.21,7.1,284.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
2,2022-01-01 02:00:00,2.71,28.3,47.1,75.2,4.0,203.0,139.31,715.4,0.0,62.0,5.9,0.0,21.21,7.7,290.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
3,2022-01-01 03:00:00,2.31,19.6,42.1,61.5,5.0,262.0,177.68,715.1,0.0,59.0,5.5,0.0,20.44,8.2,294.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
4,2022-01-01 04:00:00,1.85,13.0,39.5,52.5,4.0,185.0,134.21,715.1,0.0,59.0,4.9,0.0,19.80,8.1,287.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176


# Data Prep

In [7]:
data_joined.columns

Index(['date', 'CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'PRS', 'RAINF',
       'RH', 'SO2', 'SR', 'TOUT', 'WSR', 'WDR', 'Nombre_Estacion',
       'Clave_Estacion', 'x_coord', 'y_coord'],
      dtype='object')

In [8]:

column_map = {
    'date': 'Date',
    'x_coord': 'Latitude',
    'y_coord': 'Longitude',
    'RAINF': 'Precipitation',
    'TOUT': 'Temperature',
    'RH': 'Relative Humidity',
    'SR': 'Solar Radiation',
    'PRS': 'Atmospheric Pressure',
    'WSR': 'Wind Speed',
    'WDR': 'Wind Direction'
}

data_joined.rename(columns=column_map, inplace=True)
data_joined.replace(-9999, np.nan, inplace=True)
data_joined[data_joined.select_dtypes(include=['number']).columns] = data_joined.select_dtypes(include=['number']).interpolate(method='linear', limit_direction='both')
data_joined

,Date,CO,NO,NO2,NOX,O3,PM10,PM2.5,Atmospheric Pressure,Precipitation,Relative Humidity,SO2,Solar Radiation,Temperature,Wind Speed,Wind Direction,Nombre_Estacion,Clave_Estacion,Latitude,Longitude
0,2022-01-01 00:00:00,2.36,19.8,48.4,68.0,5.0,142.0,111.00,715.7,0.0,59.0,4.0,0.000,21.65,6.5,285.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
1,2022-01-01 01:00:00,2.29,10.8,44.5,55.1,6.0,168.0,112.16,715.5,0.0,59.0,4.6,0.000,21.21,7.1,284.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
2,2022-01-01 02:00:00,2.71,28.3,47.1,75.2,4.0,203.0,139.31,715.4,0.0,62.0,5.9,0.000,21.21,7.7,290.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
3,2022-01-01 03:00:00,2.31,19.6,42.1,61.5,5.0,262.0,177.68,715.1,0.0,59.0,5.5,0.000,20.44,8.2,294.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
4,2022-01-01 04:00:00,1.85,13.0,39.5,52.5,4.0,185.0,134.21,715.1,0.0,59.0,4.9,0.000,19.80,8.1,287.0,NORESTE3 PESQUERIA,Pesqueria,25.791343,-100.078176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205800,2023-08-17 19:00:00,0.53,3.1,8.2,11.2,40.0,108.0,17.00,705.4,0.0,60.0,3.5,0.056,36.67,14.5,66.0,SURESTE TEC NL,Tec de Nuevo León,25.668270,-100.249580
205801,2023-08-17 20:00:00,0.56,2.9,9.9,12.7,36.0,59.0,17.00,705.8,0.0,60.0,3.5,0.000,35.27,12.6,64.0,SURESTE TEC NL,Tec de Nuevo León,25.668270,-100.249580
205802,2023-08-17 21:00:00,0.57,3.0,10.3,13.2,33.0,56.0,17.00,706.3,0.0,60.0,3.0,0.000,34.57,11.6,69.0,SURESTE TEC NL,Tec de Nuevo León,25.668270,-100.249580
205803,2023-08-17 22:00:00,0.74,9.8,21.1,30.9,19.0,54.0,17.00,706.8,0.0,60.0,2.8,0.000,33.99,11.9,88.0,SURESTE TEC NL,Tec de Nuevo León,25.668270,-100.249580


In [10]:
data_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205805 entries, 0 to 205804
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Date                  205805 non-null  datetime64[ns]
 1   CO                    205805 non-null  float64       
 2   NO                    205805 non-null  float64       
 3   NO2                   205805 non-null  float64       
 4   NOX                   205805 non-null  float64       
 5   O3                    205805 non-null  float64       
 6   PM10                  205805 non-null  float64       
 7   PM2.5                 205805 non-null  float64       
 8   Atmospheric Pressure  205805 non-null  float64       
 9   Precipitation         205805 non-null  float64       
 10  Relative Humidity     205805 non-null  float64       
 11  SO2                   205805 non-null  float64       
 12  Solar Radiation       205805 non-null  float64       
 13 

In [9]:
data_joined.to_csv('dataset.csv', index=False)